In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import ollama


In [2]:
df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/data/amazon-customer-reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

In [4]:
df.shape

(28332, 24)

In [5]:
df.columns = df.columns.str.lower()

In [6]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,reviews.didpurchase,reviews.dorecommend,reviews.id,reviews.numhelpful,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
df.isnull().sum()

id                         0
dateadded                  0
dateupdated                0
name                       0
asins                      0
brand                      0
categories                 0
primarycategories          0
imageurls                  0
keys                       0
manufacturer               0
manufacturernumber         0
reviews.date               0
reviews.dateseen           0
reviews.didpurchase    28323
reviews.dorecommend    12246
reviews.id             28291
reviews.numhelpful     12217
reviews.rating             0
reviews.sourceurls         0
reviews.text               0
reviews.title              0
reviews.username           5
sourceurls                 0
dtype: int64

In [8]:
df = df.drop(columns=["reviews.didpurchase", "reviews.id", "reviews.numhelpful", "reviews.dorecommend"])

In [9]:
df.isnull().sum()

id                    0
dateadded             0
dateupdated           0
name                  0
asins                 0
brand                 0
categories            0
primarycategories     0
imageurls             0
keys                  0
manufacturer          0
manufacturernumber    0
reviews.date          0
reviews.dateseen      0
reviews.rating        0
reviews.sourceurls    0
reviews.text          0
reviews.title         0
reviews.username      5
sourceurls            0
dtype: int64

# Task 1: Review Classification

## Create sentiment categories

In [10]:
df["reviews.rating"].value_counts()

reviews.rating
5    19897
4     5648
3     1206
1      965
2      616
Name: count, dtype: int64

In [11]:
sentiment_map = {1: "negative", 2: "negative", 3: "neutral", 4: "positive", 5: "positive"}
df["rating_sentiment"] = df["reviews.rating"].map(sentiment_map)

In [12]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...",neutral
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...",positive
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...",positive
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...",positive
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...",positive


In [13]:
df["rating_sentiment"].value_counts()

rating_sentiment
positive    25545
negative     1581
neutral      1206
Name: count, dtype: int64

In [14]:
percentage_of = round((df["rating_sentiment"].value_counts() / len(df) * 100), 2)

percentage_of

# very imbalanced dataset

rating_sentiment
positive    90.16
negative     5.58
neutral      4.26
Name: count, dtype: float64

## Downsample data for balanced classes

In [15]:
# Split the dataframe into three: positive, negative, neutral
positive = (df[df["rating_sentiment"] == "positive"])
negative = (df[df["rating_sentiment"] == "negative"])
neutral =  (df[df["rating_sentiment"] == "neutral"])

positive_sampled = positive.sample(n=1206)
negative_sampled = negative.sample(n=1206)

len(positive_sampled)

# concat into one balanced dataframe 

df_balanced = pd.concat([negative_sampled, positive_sampled, neutral])

# confirm
df_balanced["rating_sentiment"].value_counts()

rating_sentiment
negative    1206
positive    1206
neutral     1206
Name: count, dtype: int64

## Choose a classifier model

- 0: Negative
- 1: Neutral
- 2: Positive

In [16]:
# pick a model 
# CardiffNLP baseline tokenizes the model
classifier = pipeline(task="sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# view a sample 

sample_reviews = df_balanced["reviews.text"].head(10).tolist()

results = classifier(sample_reviews)
results


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.8465775847434998},
 {'label': 'LABEL_2', 'score': 0.8215872645378113},
 {'label': 'LABEL_1', 'score': 0.43460822105407715},
 {'label': 'LABEL_0', 'score': 0.92408287525177},
 {'label': 'LABEL_0', 'score': 0.8691161870956421},
 {'label': 'LABEL_0', 'score': 0.9635699987411499},
 {'label': 'LABEL_0', 'score': 0.6331484317779541},
 {'label': 'LABEL_0', 'score': 0.9678872227668762},
 {'label': 'LABEL_0', 'score': 0.5253010988235474},
 {'label': 'LABEL_0', 'score': 0.6237383484840393}]

In [22]:
sample_sentiment = df_balanced["rating_sentiment"].head(10).tolist
sample_sentiment

<bound method IndexOpsMixin.tolist of 22631    negative
11286    negative
7993     negative
20705    negative
8362     negative
24005    negative
11347    negative
16890    negative
6103     negative
18956    negative
Name: rating_sentiment, dtype: object>

In [23]:
full_results = classifier(df_balanced["reviews.text"].tolist(), truncation=True, max_length=512)

In [24]:
label_strings = [r["label"] for r in full_results]
# returns a list of strings
label_strings
# turn the list of strings into a Series using pd.Series
labels_for_mapping = pd.Series(label_strings)
# now labels are ready for mapping
labels_for_mapping


0       LABEL_0
1       LABEL_0
2       LABEL_0
3       LABEL_0
4       LABEL_0
         ...   
3613    LABEL_0
3614    LABEL_0
3615    LABEL_0
3616    LABEL_0
3617    LABEL_2
Length: 3618, dtype: object

In [25]:
# map the labels into sentiment and create a new column based on the predicted sentiment
label_map = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
df_balanced["predicted_sentiment"] = labels_for_mapping.map(label_map).values

In [26]:
predicted = df_balanced["predicted_sentiment"]
actual = df_balanced["rating_sentiment"]
print(classification_report(actual, predicted))

              precision    recall  f1-score   support

    negative       0.68      0.76      0.72      1206
     neutral       0.53      0.26      0.35      1206
    positive       0.65      0.91      0.76      1206

    accuracy                           0.64      3618
   macro avg       0.62      0.64      0.61      3618
weighted avg       0.62      0.64      0.61      3618



### Export class-balanced dataset to csv for use in Colab

In [27]:
df_balanced.to_csv('balanced_reviews.csv', index=False)

# 2. Product Category Clustering

In [28]:
df["categories"]

0        AA,AAA,Health,Electronics,Health & Household,C...
1        AA,AAA,Health,Electronics,Health & Household,C...
2        AA,AAA,Health,Electronics,Health & Household,C...
3        AA,AAA,Health,Electronics,Health & Household,C...
4        AA,AAA,Health,Electronics,Health & Household,C...
                               ...                        
28327    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28328    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28329    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28330    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28331    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
Name: categories, Length: 28332, dtype: object

In [29]:
df["categories"].value_counts()

categories
AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           8343
AA,AAA,Electronics Features,Health,Electro

### Preprocess the text for embedding and clustering

In [46]:
# split and explode categores to produce a series of words
categories = df["categories"].str.split(",").explode()
categories.tolist()

['AA',
 'AAA',
 'Health',
 'Electronics',
 'Health & Household',
 'Camcorder Batteries',
 'Camera & Photo',
 'Batteries',
 'Household Batteries',
 'Robot Check',
 'Accessories',
 'Camera Batteries',
 'Health and Beauty',
 'Household Supplies',
 'Batteries & Chargers',
 'Health',
 ' Household & Baby Care',
 'Health Personal Care',
 'AA',
 'AAA',
 'Health',
 'Electronics',
 'Health & Household',
 'Camcorder Batteries',
 'Camera & Photo',
 'Batteries',
 'Household Batteries',
 'Robot Check',
 'Accessories',
 'Camera Batteries',
 'Health and Beauty',
 'Household Supplies',
 'Batteries & Chargers',
 'Health',
 ' Household & Baby Care',
 'Health Personal Care',
 'AA',
 'AAA',
 'Health',
 'Electronics',
 'Health & Household',
 'Camcorder Batteries',
 'Camera & Photo',
 'Batteries',
 'Household Batteries',
 'Robot Check',
 'Accessories',
 'Camera Batteries',
 'Health and Beauty',
 'Household Supplies',
 'Batteries & Chargers',
 'Health',
 ' Household & Baby Care',
 'Health Personal Care',
 'AA

In [31]:
# remove the top 10 words from the series 
# the words overpower each cluster, causing redundencies
top_10words = categories.value_counts().head(10)
remove_terms = top_10words.index.tolist()
remove_terms


['Electronics',
 'Health',
 'Computers & Tablets',
 'Tablets',
 'Fire Tablets',
 'Amazon Tablets',
 'Accessories',
 'AA',
 'Household Supplies',
 'AAA']

In [32]:
# goal: replace the categories with a new category list minus removed words

df["categories_clean"] = df["categories"] # copy category list to new column

for term in remove_terms: # for each word in the list 
    df["categories_clean"] = df["categories_clean"].str.replace(term, "", regex=False) # replace term with blank

# now we can vectorize the clean categories column

## Vectorize the text to prep for modeling

In [33]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

text = df["categories_clean"].str.strip().str.lower()

embeddings_cleaned = model.encode(text)

In [34]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)

kmeans.fit(embeddings_cleaned)

labels = kmeans.labels_

df["cluster_cleaned"] = labels

df["cluster_cleaned"].value_counts()

cluster_cleaned
1    13492
2     8345
0     3728
3     2767
Name: count, dtype: int64

In [35]:
for i in range(kmeans.n_clusters):
    print(f"\n=== Cluster {i} ===")
    print(df[df["cluster_cleaned"] == i]["categories"].head(10).tolist())



=== Cluster 0 ===
['AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Hea

In [36]:
batteries_and_cameras = df[df["cluster_cleaned"] == 0]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)


In [37]:
electronics = df[df["cluster_cleaned"] == 1]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)


In [39]:
household_beauty = df[df["cluster_cleaned"] == 2]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)
print(f"---Cluster 2: Household & Personal---")
print()

---Cluster 2: Household & Personal---



In [ ]:
kids = df[df["cluster_cleaned"] == 3]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)

categories_clean
          5467
Fire      2767
Amazon    2767
All       2474
Kids'      293
Name: count, dtype: int64

## Ollama

In [17]:
sample = df["categories"].iloc[0]
len(sample) # one category for each product

259

In [18]:
# goal: 

df["primarycategories"].iloc[57]

'Health & Beauty'

In [19]:
df["primarycategories"].value_counts()

primarycategories
Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Home & Garden                      2
Electronics,Furniture              2
Name: count, dtype: int64

In [20]:
office_supplies = df[df["primarycategories"] == "Office Supplies"]
office_supplies

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
12156,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-02T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,This folder also expands on the bottom. Will a...,Bottom expands as well,happypersonohyeh,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12157,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2019-03-25T00:00:00.000Z,"2019-04-22T00:00:00Z,2019-04-19T00:00:00Z",5,https://www.ebay.com/itm/Expanding-Accordion-F...,Exactly what I needed.,Folder,99kurama99,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12158,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-11-09T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/itm/Expanding-File-Tabs-F...,Nice product. Will work perfectly for my proje...,Folder,juleekae,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12159,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-27T00:00:00.000Z,"2017-10-30T00:00:00Z,2017-09-21T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,A useful folder organizer with good quality.,Good purchase,ml2shop,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12160,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-01-12T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,I love the portability of this expandable file...,Great product and cost,connieinla,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12161,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2018-06-12T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/urw/product-reviews/10014...,Exactly what I needed!!!!,Perfect!,lobo5981,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12162,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder..

In [21]:
df["primarycategories"]

0        Health & Beauty
1        Health & Beauty
2        Health & Beauty
3        Health & Beauty
4        Health & Beauty
              ...       
28327        Electronics
28328        Electronics
28329        Electronics
28330        Electronics
28331        Electronics
Name: primarycategories, Length: 28332, dtype: object

In [22]:
unique_categories = df["categories"].unique()
unique_categories.tolist()

['AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care',
 'Audio & Video Accessories,TV, Video & Home Audio,Home Audio & Video Accessories,DVD Cases,Electronics,Media Cases & Storage,TV, Video & Audio Accessories,Media Storage & Organization,Disc Storage Wallets,Consumer Electronics,CD, DVD & Blu-ray Discs,Blank Audio/Video Media,Accessories & Supplies,all electronics',
 'Stereos,Remote Controls,Audio Docks & Mini Speakers,Kitchen & Dining Features,Home & Tools,Electronics,New Networking,Wireless Speakers,Home, Garage & Office,Portable Speakers,iPod, Audio Player Accessories,Smart Home,Surveillance,Home Improvement,Smart Home & Home Automation Devices,Alarms & Sensors,Amazon Devices,Amazon,Holiday Shop,Kitchen Dining Features,Amazon Echo,Amazon Echo Accessories,Networking,Speaker Sy

In [23]:
category_map = {}

for cat in unique_categories:
    response = ollama.chat(model="qwen2.5", messages=[
        {"role": "user", "content": f"""Classify this product category: '{cat}' into ONE of: 'Electronics', 'Health & Beauty', 'Tablets & E-readers', 'Home & Kitchen', 'Office Supplies', 'Pet Supplies'. Reply with ONLY the category name. 
         DO NOT invent your own category name. Use ONLY the names provided.
         If the product category contains Health & Beauty or Health and Beauty, then the product category is: Health & Beauty
         Litter boxes are classified as Pet Supplies.
         
         """}
    ])
    category_map[cat] = response["message"]["content"]

In [24]:
# turn the dictionary into a dataframe 
# use .items() to split the key:value pairs into a list of tuples so pd.Dataframe can create two colums, key and value

mapping_df = pd.DataFrame(category_map.items(), columns=["categories", "primary_category"])
mapping_df.head()

,categories,primary_category
0,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty
1,"Audio & Video Accessories,TV, Video & Home Aud...",Electronics
2,"Stereos,Remote Controls,Audio Docks & Mini Spe...",Electronics
3,"Amazon Echo,Virtual Assistant Speakers,Electro...",Electronics
4,"Amazon Echo,Networking,Home & Tools,Smart Home...",Electronics


In [25]:
df["meta_category"] = df["categories"].map(category_map)

In [26]:

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df[["categories", "primarycategories", "meta_category"]].value_counts()

categories                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  primarycategories            meta_category      
AA,AAA,Heal

In [27]:
df.columns

Index(['id', 'dateadded', 'dateupdated', 'name', 'asins', 'brand',
       'categories', 'primarycategories', 'imageurls', 'keys', 'manufacturer',
       'manufacturernumber', 'reviews.date', 'reviews.dateseen',
       'reviews.rating', 'reviews.sourceurls', 'reviews.text', 'reviews.title',
       'reviews.username', 'sourceurls', 'rating_sentiment', 'meta_category'],
      dtype='object')

## Save the df with meta_categories and rating_sentiment labels as a csv to save tokens & time when reloading

In [28]:
df.to_csv("amazon_sentiment_categories.csv", index=False)

In [3]:
new_df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/notebooks/amazon_sentiment_categories.csv")

# Task 3: Review Summarization Using Generative AI

In [14]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
new_df.groupby(['meta_category', 'name'])['reviews.rating'].mean().sort_values()

meta_category        name                                                                                                                                                  
Tablets & E-readers  Oem Amazon Kindle Power Usb Adapter Wall Travel Charger Fire/dx/+micro Usb Cable                                                                          1.000000
Electronics          Certified Refurbished Amazon Fire TV with Alexa Voice Remote                                                                                              2.800000
                     Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only                                        2.800000
Pet Supplies         AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)                                                                               3.500000
Home & Kitchen       AmazonBasics Silicone Hot Handle Cover/Holder - Red                    

In [4]:
electronics = new_df[new_df["meta_category"] == "Electronics"]


In [72]:
top_3_e = electronics.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_e

avg_rating  \
meta_category name                                                                                            
Electronics   Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers    4.597217   
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case               4.531026   
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case               4.534035   

                                                                                                 num_reviews  
meta_category name                                                                                            
Electronics   Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers         2443  
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case                    1676  
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case                    1425

In [75]:
e_1 = new_df[new_df["name"] == "Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers"]["reviews.text"]
e_1.to_list()

['While there are some apps that are missing due to not using the Google Play Store the list is minimal. The tablet itself is quick, responsive, and noticeable nicer than the 7" fire. Picked this up when it was on sale for black friday sales. It\'s great for in front of the TV and catching up on email and light web browsing.',
 'I bought one last year for one cousin and had to continue for another this year. Great buy imo.',
 'I will try again for the 3rd time with my new Amazon Fire. It seems to have nice upgrades.',
 "bought this for my father, who never did much on computers and was not sure he would actually use this. He unwrapped it, asked for some games to be downloaded and hasn't looked back! He has used the internet as well as the game apps and couldn't be happier.",
 'This was a request from our 7 yr old granddaugter who had always asked her older sister if she could share hers due to we bought this same tablet/size a few yrs earlier. Loved the security/parental control option

In [76]:
e_2 = new_df[new_df["name"] == "Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case"]["reviews.text"]
e_2.to_list()

['great tablet for kids when traveling. my grandchild loves it.',
 'Bought this for my 2 yr old daughter. She was able to use it within minutes. She loves bed it, plays games on it everyday as well as we bought apps to read to her. She loves it all.',
 "Kindle products are fantastic. I've used a kindle & now have a Kindle fire. Use it everyday, so naturally, I would select a Kindle for my granddaughter.",
 "Best buy ever for my 3 year old. Only purchased because they were out of the Galaxy tablet. I'm glad. This tablet is PERFECT for my 3 year old. I love that it has 2 modes. The kids mode sometimes freeze and takes a minutes for apps to open. Also didn't download Netflix movies like indicated. But regular mode works great. My daughter loves watching weird homemade youtube movies anyway.",
 "My little girl loves her Kindle. I'm so happy I bought an upgrade to her previous kindle. Best Christmas gift ever.",
 "My Grandson can't put it down! With access to so many educational apps and ga

In [77]:
e_3 = new_df[new_df["name"] == "Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case"]["reviews.text"]
e_3.to_list()

['great tablet for kids when traveling. my grandchild loves it.',
 'Bought this for my 2 yr old daughter. She was able to use it within minutes. She loves bed it, plays games on it everyday as well as we bought apps to read to her. She loves it all.',
 "Kindle products are fantastic. I've used a kindle & now have a Kindle fire. Use it everyday, so naturally, I would select a Kindle for my granddaughter.",
 "My little girl loves her Kindle. I'm so happy I bought an upgrade to her previous kindle. Best Christmas gift ever.",
 'My kids have grown up with technology in the house and their original tablets have started to become outdated. We have since switched to these fire tablets to help extend their gaming. While if they have never had a tablet before the speeds are OK but Im afraid the processing power of these "kids" versions might lag over a short amount of time and they will just become a picture viewer. Time will only tell.',
 "This is the second Kid's version of the Fire that I've

In [69]:
worst_e_product = electronics.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_e_product

,,avg_rating,num_reviews
meta_category,name,,
Electronics,Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only,2.8,5


In [61]:
electronic_reviews_low = new_df[new_df["name"] == "Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only"]["reviews.text"]

In [66]:
electronic_reviews_low.to_list()

['Since the details for the items are a little sparse I thought I would provide the specs listed on the power adapter.AC Input 100-240 Volts, 0.15 Amps, 50/60 HzDC Output 4.9 Volts, 0.85 AmpsThis adapter has a Flextronics part number of 09500043-200, Model A00810',
 'I was initially happy that my Kindle replacement charging cord arrived so promptly, it seemed to work fine. But after only one use, the outer covering of the cord started cracking falling off in bits. It now is almost completely covered by electrical tape, as the bits fall off w/ each every use. I will end up having to buy a new one after all...',
 "I bought the Kindle when I was in the USA but on my return to Australia I discovered I left the power adapter behind in my condo in the USA. For some reason Amazon cannot or will not mail a replacement cable to Australia so now I'm unable to re-charge my Kindle as the Micro USB cables available in Australia do not fit the Kindle as it is tiny compared to the ones available here

In [82]:
# goal, give qwen the names, raings, and reviews, then ask

summaries = {}

response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article (like a blog post) the product category. The output should include:

Top 3 products {top_3_e} and key differences between them {e_1}, {e_2}, {e_3}.
Top complaints for each of those products.

----- 
You should also include the worst product {worst_e_product} in the category and why it should be avoided {electronic_reviews_low}.
Do not forget to include this.
----

Make sure the style is like a blog.

""" }

])

summaries["electronics"] = response['message']['content']

In [94]:
print(response['message']['content'])

### Top 3 Best-Selling Amazon Fire Tablets and Key Differences

When it comes to finding the perfect tablet for your family or personal use, Amazon's Fire line offers some of the best options on the market. Let’s dive into the top three best-selling products and explore what makes them stand out.

#### 1. **Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine**
   - **Average Rating:** 4.597
   - **Number of Reviews:** 2,443

**Features:**
- Equipped with a large 8-inch HD display.
- Comes pre-installed with Alexa for voice control and smart home integration.
- Offers plenty of storage space (16 GB) to store your favorite apps, games, and media.

**Pros:**
- Quick and responsive performance.
- Great for streaming videos, browsing the web, and managing emails.
- Affordable at just under $100.

**Cons:**
- Some users report minor lag in performance, but this is generally not a significant issue.

#### 2. **Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case**
  

In [86]:
electronic_blog_summary = summaries["electronics"]

In [89]:
import json

with open("summaries.json", "w") as f:
    json.dump(summaries, f)

In [91]:
with open("summaries.json", "r") as f:
    summaries = json.load(f)

In [92]:
summaries

{'electronics': "### Top 3 Best-Selling Amazon Fire Tablets and Key Differences\n\nWhen it comes to finding the perfect tablet for your family or personal use, Amazon's Fire line offers some of the best options on the market. Let’s dive into the top three best-selling products and explore what makes them stand out.\n\n#### 1. **Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine**\n   - **Average Rating:** 4.597\n   - **Number of Reviews:** 2,443\n\n**Features:**\n- Equipped with a large 8-inch HD display.\n- Comes pre-installed with Alexa for voice control and smart home integration.\n- Offers plenty of storage space (16 GB) to store your favorite apps, games, and media.\n\n**Pros:**\n- Quick and responsive performance.\n- Great for streaming videos, browsing the web, and managing emails.\n- Affordable at just under $100.\n\n**Cons:**\n- Some users report minor lag in performance, but this is generally not a significant issue.\n\n#### 2. **Fire Kids Edition Tablet, 7 Display, 